## ROC AUC calculation in sklearn: the trick is to one-hot encode the targets

In [3]:
import numpy as np
np.random.seed(0)

from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics as skm

In [57]:
def sklearn_one_hot(a):
    b = np.zeros((a.size, a.max()+1))
    b[np.arange(a.size),a] = 1
    return b

def make_multi_label():
    _, y1 = datasets.make_classification(n_classes=5, n_samples=10000, n_features=20, n_informative=5, n_redundant=2)
    X, y2 = datasets.make_classification(n_classes=5, n_samples=10000, n_features=20, n_informative=5, n_redundant=2)
    return X, (sklearn_one_hot(y1) + sklearn_one_hot(y2)).clip(max=1.)

### Binary case

In [17]:
X, y = datasets.make_classification(n_classes=2, n_samples=10000, n_features=20, n_informative=2, n_redundant=2)
X.shape, y.shape

((10000, 20), (10000,))

In [26]:
clf = RandomForestClassifier(min_samples_leaf=100)
clf.fit(X, y);

In [27]:
probas, targets = clf.predict_proba(X), y
probas.shape, targets.shape, sklearn_one_hot(targets).shape

((10000, 2), (10000,), (10000, 2))

In [28]:
sklearn_one_hot(targets)[:5], targets[:5]

(array([[0., 1.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [1., 0.]]),
 array([1, 0, 0, 1, 0]))

In [29]:
skm.roc_auc_score(sklearn_one_hot(targets), probas)

0.9850665050665051

### Multi-class case

In [88]:
X, y = datasets.make_classification(n_classes=5, n_samples=10000, n_features=20, n_informative=5, n_redundant=2)
X.shape, y.shape

((10000, 20), (10000,))

In [89]:
clf = RandomForestClassifier(min_samples_leaf=100)
clf.fit(X, y);

In [90]:
probas, targets = clf.predict_proba(X), y
probas.shape, targets.shape, sklearn_one_hot(targets).shape

((10000, 5), (10000,), (10000, 5))

In [91]:
sklearn_one_hot(targets)[:5], targets[:5]

(array([[0., 0., 0., 0., 1.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.]]),
 array([4, 2, 1, 3, 2]))

In [97]:
skm.roc_auc_score(sklearn_one_hot(targets), probas)

0.9290266500562648

In [98]:
skm.roc_auc_score(targets, probas, multi_class='ovr')

0.9290266500562648

In [100]:
skm.roc_auc_score(targets, probas, multi_class='ovo')

0.9290257127202818

### Multi-label multi-class case

In [58]:
X, y = make_multi_label()
X.shape, y.shape

((10000, 20), (10000, 5))

In [59]:
clf = RandomForestClassifier(min_samples_leaf=100)
clf.fit(X, y);

In [83]:
probas, targets = clf.predict_proba(X), y

probas = [proba[:,1] for proba in probas]
probas = np.vstack(p_).T

probas.shape, targets.shape

((10000, 5), (10000, 5))

In [86]:
targets[:5]

array([[0., 1., 1., 0., 0.],
       [0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 1.],
       [0., 1., 0., 1., 0.],
       [0., 1., 0., 1., 0.]])

In [87]:
skm.roc_auc_score(targets, probas)

0.7556312398744894

## Proposal to fix [RocAuc](http://dev.fast.ai/metrics#RocAuc) in fast.ai 

* **Binary case**:
    * `targets`: one-hot-encode, shape = `(n_samples, 2)` 
    * `preds`: pass through **sigmoid**, shape = `(n_samples, 2)`
    
    
* **Multi-class case**:
    * `targets`: one-hot-encode, shape = `(n_samples, n_classes)` 
    * `preds`: pass through **softmax**, shape = `(n_samples, n_classes)`
    
    
* **Multi-label Multi-class case**:
    * `targets`: one-hot-encode, shape = `(n_samples, n_classes)` 
    * `preds`: pass through **sigmoid**, shape = `(n_samples, n_classes)`